As you might now, letsencrypt ssl certificates officially reached production state, see  https://community.letsencrypt.org/t/acme-v2-production-environment-wildcards/55578 .

It is time to give it a try. I have two DNS providers in use: Godaddy and Cloudflare, thus I had two use to different methods to make things happen.
At a moment of article writing, certbot was not supporting latest features, thus I had to install latest from sources:

```sh
git clone git@github.com:certbot/certbot.git && cd certbot
sudo python setup.py install
```

Wildcard certificates require dns validation. Thus, once you installed certbot, look for plugins that support your DNS provider.

In my case there was cloudflare plugin, thus for that domain I was able to achieve fully automatic generation/renewal

In [1]:
%%bash
certbot plugins


-------------------------------------------------------------------------------
* standalone
Description: Spin up a temporary webserver
Interfaces: IAuthenticator, IPlugin
Entry point: standalone = certbot.plugins.standalone:Authenticator

* webroot
Description: Place files in webroot directory
Interfaces: IAuthenticator, IPlugin
Entry point: webroot = certbot.plugins.webroot:Authenticator
-------------------------------------------------------------------------------


```sh
ls | grep certbot-dns
certbot-dns-cloudflare
certbot-dns-cloudxns
certbot-dns-digitalocean
certbot-dns-dnsimple
certbot-dns-dnsmadeeasy
certbot-dns-google
certbot-dns-luadns
certbot-dns-nsone
certbot-dns-rfc2136
certbot-dns-route53
```

```sh
cd certbot-dns-cloudflare 
sudo python setup.py install

```

## Clould flare installation

To use the authenticator plugin with CloudFlare, you need to provide CloudFlare api key to the bot so it can edit the domain entries to add validation TXT entry to verify you control the domain. This of course means, that you should take care on configuration files with key.


You obtain the Global API key in CloudFlare from your user profile,

Put these keys into a configuration file. Certbot uses a default directory`/etc/letsencrypt`.  We need to create file 
`/etc/letsencrypt/dnscloudflare.ini` to store credentials from CloudFlare.  

```
# CloudFlare API key information
dns_cloudflare_api_key = yourcloudflarekey
dns_cloudflare_email = yourcloudflarelogin
```

Ensure file is readable only by allowed persons, saying root.

```sh
chmod 600 /etc/letsencrypt/dnscloudflare.ini
```

## Certbot Configuration Settings
Wildcard certificates are only available via the v2 API, which I haven't found in certbot installed from packages, so I had to amend configuration to tell certbot server parameter. Certbot uses the `/etc/letsencrypt/cli.ini`
```
# Let's Encrypt site-wide configuration
dns-cloudflare-credentials = /etc/letsencrypt/dnscloudflare.ini
# Use the ACME v2 staging URI for testing things
#server = https://acme-staging-v02.api.letsencrypt.org/directory
# Production ACME v2 API endpoint
server = https://acme-v02.api.letsencrypt.org/directory
```

## Generating certificate in automatic mode with ClouldFlare pluging

```sh
sudo certbot certonly -d *.voronenko.info --dns-cloudflare --agree-tos
Saving debug log to /var/log/letsencrypt/letsencrypt.log
Plugins selected: Authenticator dns-cloudflare, Installer None
Starting new HTTPS connection (1): acme-v02.api.letsencrypt.org

Obtaining a new certificate
Performing the following challenges:
dns-01 challenge for voronenko.info
Starting new HTTPS connection (1): api.cloudflare.com
Waiting 10 seconds for DNS changes to propagate
Waiting for verification...
Cleaning up challenges
Starting new HTTPS connection (1): api.cloudflare.com

IMPORTANT NOTES:
 - Congratulations! Your certificate and chain have been saved at:
   /etc/letsencrypt/live/voronenko.info/fullchain.pem
   Your key file has been saved at:
   /etc/letsencrypt/live/voronenko.info/privkey.pem
   Your cert will expire on 2018-06-19. To obtain a new or tweaked
   version of this certificate in the future, simply run certbot
   again. To non-interactively renew *all* of your certificates, run
   "certbot renew"

```

## Case 2: not supported DNS provider

I have also another domain, used for open source activities. For that domain I had to add validation entry manually.
Command is slightly different, note that most of -- parameters might go config files. If you have improvements, comments are welcomed.

```sh
certbot certonly --manual -d *.softasap.com --agree-tos --no-bootstrap --manual-public-ip-logging-ok --preferred-challenges dns-01 --server https://acme-v02.api.letsencrypt.org/directory
```

During the command run you are asked to put TXT entry into DNS records, and wait for change to propagate

```sh
sudo certbot certonly --manual -d *.softasap.com --agree-tos --no-bootstrap --manual-public-ip-logging-ok --preferred-challenges dns-01 --server https://acme-v02.api.letsencrypt.org/directory
Saving debug log to /var/log/letsencrypt/letsencrypt.log
Plugins selected: Authenticator manual, Installer None
Starting new HTTPS connection (1): acme-v02.api.letsencrypt.org
Obtaining a new certificate
Performing the following challenges:
dns-01 challenge for softasap.com

-------------------------------------------------------------------------------
Please deploy a DNS TXT record under the name
_acme-challenge.softasap.com with the following value:

12KCzU4bv-FTzjeFZi8zIaEUm049VyHv1DIApg78nmY

Before continuing, verify the record is deployed.
-------------------------------------------------------------------------------
Press Enter to Continue
Waiting for verification...
Cleaning up challenges

IMPORTANT NOTES:
 - Congratulations! Your certificate and chain have been saved at:
   /etc/letsencrypt/live/softasap.com/fullchain.pem
   Your key file has been saved at:
   /etc/letsencrypt/live/softasap.com/privkey.pem
   Your cert will expire on 2018-06-19. To obtain a new or tweaked
   version of this certificate in the future, simply run certbot
   again. To non-interactively renew *all* of your certificates, run
   "certbot renew"
```

## Summary

We've successfully used new letsencrypt API go generate wildcard certificate in fully automated mode as well as in manual mode